### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현


In [4]:
from pprint import pprint
from langchain.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import WikipediaLoader
from typing import List
from langchain_community.vectorstores import FAISS
import re
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 


# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = [] # [Document, Document]
for doc in documents:
    menu_documents += split_menu_items(doc)

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

menu_db.save_local("../db/cafe_db")



In [5]:
from langchain_core.tools import tool
from langchain_community.tools import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.prompts import ChatPromptTemplate

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def tavily_search_func(query: str) -> str:
    """
        Searches the internet for information that does not exist in the database or for the latest information.
        Use this for up-to-date information, news, or general web searches (e.g., '최신 커피 트렌드', '카페 위치 정보').
    """
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

@tool
def wiki_summary(query:str)-> str:
    """ 
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.(e.g., '커피 역사', '음료 제조 방법').
    """
    wiki_loader = WikipediaLoader(query=query, load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()
    
    if wiki_docs:
        return wiki_docs
    else:
        return "위키피디아에서 관련 정보를 찾을 수 없습니다."

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
#     temperature=0.2
# )

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary, db_search_cafe_func])

prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm_chain = prompt | llm_with_tools

@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config) 
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)   

    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 도구 호출이 필요한 LLM 호출을 수행
query = "아메리카노의 가격과 특징은 무엇인가요?"
response = cafe_menu_chain.invoke(query)

# 응답 출력 
print(response.content)

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.12\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.12\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


아메리카노는 에스프레소에 뜨거운 물을 추가하여 만드는 커피로, 그 맛은 에스프레소보다 부드럽고 연합니다. 아메리카노의 이름은 '미국의 커피'를 의미하며, 특히 2차 세계 대전 때 미국 군인들이 이탈리아의 강한 에스프레소를 연하게 하기 위해 물을 섞어 마신 데서 유래되었습니다.

다음은 아메리카노의 특징과 가격에 대한 요약입니다:

### 특징:
- **조리 방법**: 에스프레소에 따뜻한 물을 추가하여 만든다.
- **맛**: 에스프레소보다 부드럽고 연하며, 커피의 풍미가 강하지 않다.
- **유래**: 'Caffè Americano'라는 이름은 영어로 'American coffee'로 번역되며, 미국 커피를 나타낸다. 
- **온도**: 일반적으로 뜨겁게 제공되지만, 아이스 아메리카노와 같은 변형도 존재한다.

### 가격:
- 아메리카노의 가격은 일반적으로 약 4,500원에서 7,000원까지 다양합니다. 이는 카페의 종류나 위치에 따라 다를 수 있습니다. 예를 들어:
  - 일반 카페 아메리카노: 약 ₩4,500
  - 프리미엄 카페 아메리카노: 약 ₩5,000 이상

이외에도 아메리카노는 아이스 버전으로 제공되기도 하며, 다양한 커피 음료와 조화를 이루는 인기 있는 선택입니다.


### 문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트


In [6]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    HumanMessage("아메리카노 정보와 커피 역사를 알려주세요.", name="example_user"),
    AIMessage("메뉴 검색과 위키피디아 검색을 진행하겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}]),
    ToolMessage(
        "아메리카노\n• 가격: ₩4,500\n• 주요 원료: 에스프레소, 뜨거운 물\n• 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.",
        tool_call_id="1"
    ), 
    AIMessage("아메리카노 정보에 대해 알아보았습니다. 이제 커피 역사를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "커피 역사"}, "id": "2"}]),
    ToolMessage(
        "커피는 9세기경 에티오피아에서 시작되어 아라비아반도를 거쳐 전 세계로 퍼져나갔습니다. 초기에는 열매를 씹거나 삶아 먹는 방식이었으나, 점차 로스팅하여 음료로 마시는 형태로 발전했습니다. 유럽에는 17세기경 전파되어 널리 사랑받게 되었습니다.",
        tool_call_id="2"
    ),
    AIMessage("아메리카노(₩4,500)는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피로, 원두 본연의 맛을 가장 잘 느낄 수 있습니다. 커피는 9세기경 에티오피아에서 시작되어 전 세계로 퍼져나갔으며, 점차 로스팅하여 음료로 마시는 형태로 발전했습니다.", name="example_assistant"),
]


system = """
당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.
도구 사용 가이드라인:
- db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명)
- wiki_summary: 일반 지식 (역사, 제조법, 문화적 배경)  
- tavily_search_func: 최신 정보 (트렌드, 뉴스, 실시간 정보)

사용 원칙:
1. 카페 메뉴 관련 질문 → 반드시 메뉴 DB 먼저 검색
2. 역사/문화/일반 지식 → 위키피디아 활용
3. 최신 트렌드/뉴스 → 웹 검색 활용
4. 복합 질문 → 여러 도구 순차 사용
5. 정보 출처를 명확히 구분하여 답변"

효과: AI가 상황에 맞는 올바른 도구를 선택하도록 유도
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
#     temperature=0.2
# )
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary, db_search_cafe_func])

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config) 
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)   

    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래에 대해서도 알려주세요."
response = cafe_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

('카페라떼 정보는 다음과 같습니다:\n'
 '- **가격**: ₩5,500\n'
 '- **주요 원료**: 에스프레소, 스팀 밀크\n'
 '- **설명**: 크리미한 스팀 밀크를 추가한 커피로, 부드럽고 진한 맛을 느낄 수 있습니다. 다양한 맛의 시럽이나 토핑을 추가할 수 '
 '있습니다.\n'
 '\n'
 '카페라떼와 잘 어울리는 디저트는 크루아상, 티라미수, 그리고 패스츄리와 같은 부드럽고 크리미한 디저트입니다.\n'
 '\n'
 '### 카페라떼의 유래\n'
 '카페라떼(caffè latte)는 이탈리아어로 "우유와 커피"를 의미합니다. 커피와 우유를 섞은 형태로, 20세기 초 이탈리아에서 시작된 '
 '것으로 알려져 있습니다. 특히 미국에서 보편화되었으며, 바리스타들이 에스프레소에 스팀 밀크를 더해 제공함으로써 현대의 형태로 '
 '발전하였습니다.랍 확장한 스타일이 자주 즐겨지며, 카페문화와 함께 성장해 왔습니다.')
